## Term 3 

## Behavioral Planning

The behavior planning team is responsible for providing guidance to the trajectory planner about what sorts of maneuvers they should plan trajectories for. If you think about the over all flow of data in a self-driving car:

* operating on the fastest time scales you have control. 
* And with a slightly lower frequency than that you have Sensor Fusion. 
* Just lower than that you have localization and trajectory planning which you learn more about in a next lesson. 
* Next is Prediction which you just learned about. 
* And then at the top of this diagram is behavior planning with the lowest update rate. 

<img src="./outputs/bp1.png" height=600 width=600 />

The inputs to behavior planning come from the **prediction module** and the **localization module**. Both of which get their inputs from the **sensor fusion**. And the **output** from the behavior module goes directly to the **trajectory planner**. Which also takes **input** from prediction and localization so that it can send trajectories to the **motion controller**. Everything inside of marked the box is the focus of this course and is commonly referred to as Path Planning. In this lesson, you will learn what exactly has to happen in this comparatively long time span. But intuitively this longtime span comes from the fact that the behavior plan has to incorporate a lot of data to make decisions about fairly long time horizons in the order of 10 seconds or even more. We will start by doing a brief introduction where you'll learn more about the details of the inputs and outputs to behavior planner and understand what problems a behavior planning module is expected to solve. Next, we will talk about Finite State Machines as one technique for implementing a behavior plan. Followed by a more in-depth look at the Cost Functions we will use to actually make behavioral level decisions. But first let's spend some time looking at the inputs we would be working with and the outputs we will be generating.

<img src="./outputs/bp2.png" height=300 width=300 />

## Understanding Output

It's possible to suggest a wide variety of behaviors by specifying only a few quantities. For example by specifying only a target lane, a target vehicle (to follow), a target speed, and a time to reach these targets, we can make suggestions as nuanced as "stay in your lane but get behind that vehicle in the right lane so that you can pass it when the gap gets big enough."

Look at the picture below and 5 potential <span style="color:red">json</span> representations of output and see if you can match the <span style="color:red">json </span> representation with the corresponding verbal suggestion.

<img src="./outputs/bp3.jpg" height=600 width=600 />


```
Output A
{
    "target_lane_id" : 2,
    "target_leading_vehicle_id": 3,
    "target_speed" : null,
    "seconds_to_reach_target" : null,
}
Output B
{
    "target_lane_id" : 3,
    "target_leading_vehicle_id": null,
    "target_speed" : 20.0,
    "seconds_to_reach_target" : 5.0,
}
Output C
{
    "target_lane_id" : 2,
    "target_leading_vehicle_id": null,
    "target_speed" : 15.0,
    "seconds_to_reach_target" : 10.0,
}
Output D
{
    "target_lane_id" : 2,
    "target_leading_vehicle_id": 2,
    "target_speed" : null,
    "seconds_to_reach_target" : 5.0,
}
Output E
{
    "target_lane_id" : 1,
    "target_leading_vehicle_id": 2,
    "target_speed" : null,
    "seconds_to_reach_target" : 5.0,
}
```

<img src="./outputs/bp4.png" />

## The Behavior Problem 

Imagine you're driving in a city with your friend. You have a goal that you're trying to get to. You are sitting in the passenger seat and your friend is driving. You plug that goal into Google Maps and you get some route that will get you to your destination. The driver shouldn't be concerned about the details of what the route exactly is because the driver trusts that you acting, as the navigator, will tell them what maneuvers they need to make and it's understood that they are just responsible for executing them. 

In this situation, the responsibilities of the passenger are analogous to the responsibilities of a behavior planner in a self-driving car. So if you tell your friend to go left, they will do that. And when you tell the driver to pass a slow vehicle, the driver won't worry about whether they have enough time to do so because they will assume that you have thought about that and that you're confident that they can make it. But the navigator is not responsible for safety. If you tell the driver to get back to the left lane but there is a vehicle in the way, you will assume that he will wait to move left until there is enough room to do so safely. The navigator will also tell the driver when to keep their lane and when it is time to turn or change lanes or maybe adjust the speed. And finally, once you're arrived at your destination, you will tell the driver to stop. 

So, to summarize, the behavior planner is currently a black box which takes as input:
* a map of the world, 
* a route to the destinations, 
* and predictions about what other static and dynamic obstacles are likely to do, 

and it produces as output 
* a adjusted maneuver for the vehicle which the trajectory planner is responsible for reaching collision-free, smooth, and safe.

<img src="./outputs/bp5.png" height=600 width=600 />

The goal of this lesson is to open up this black box and learn how to implement a behavior planner. The **responsibilities of the behavior module are to suggest maneuvers** which are:

* feasible, 
* as safe as possible, 
* legal, 
* and efficient. 

What the behavior planner is **not responsible** for are:
* execution details 
* and collision avoidance. 

The approach to behavior planning we are going to teach in this lesson is the same one used by most of the vehicles, including the winning one in the DARPA Urban Challenge. In fact, it's the approach to behavior planning that Mercedes used in their Bertha Benz Drive, which you can see is capable of handling complex traffic situations like navigating intersections and dense urban traffic. And while this approach is not necessarily the most common anymore, for reasons we will discuss later, it is widely known in the field and it does give a very high-level understanding of what is going on in a behavior planner. 

## Finite State Machine

In the previous administration the navigator gave various suggestions on what to do next. And the instructions they gave were about things like changing lanes, following lanes, turning and so on. But in fact there really aren't that many types of suggestions you would expect to hear from a navigator. In this lesson we're going to teach an approach to behavior planning that uses something called a Finite State Machine to solve the behavior planning problem. A Finite State Machine makes decisions based on a finite set of discrete states. In this example five. When initialized, a Finite State Machine begins in some start state, let's call it S Zero. Any pair of states within the finite state machine can be connected by one or more transitions. And sometimes there is a transition back to the same state. This is called a Self Transition. Not all transitions are necessarily possible. For example, S four doesn't transition to any other state. In the language of finite state machines this would be called an Accepting State. For non-accepting states that can often be multiple potential successors states. To decide which state to transition to next, a Finite State Machine needs to handle some sort of input and then use a state transition function to decide what state to go next. 

<img src="./outputs/bp6.png" height=600 width=600/>


We'll talk more about transition functions and the states associated with a self driving car in a minute. But first, we want to formalize what a Finite State Machine is, with the help of a simple example. 

## Notes on FSM

### [Wikipedia](https://en.wikipedia.org/wiki/Finite-state_machine)

A finite-state machine (FSM) or finite-state automaton or simply a state machine, is a mathematical model of computation. It is an abstract machine that **can be in exactly one of a finite number of states at any given time**. The FSM can change from one state to another in response to some external inputs; the change from one state to another is called a transition. An FSM is defined by a list of its states, its initial state, and the conditions for each transition. Finite state machines are of two types - deterministic finite state machines and non-deterministic finite state machines.

The behavior of state machines can be observed in many devices in modern society that perform a predetermined sequence of actions depending on a sequence of events with which they are presented. Examples are vending machines, which dispense products when the proper combination of coins is deposited, elevators, whose sequence of stops is determined by the floors requested by riders, traffic lights, which change sequence when cars are waiting, and combination locks, which require the input of combination numbers in the proper order.

The finite state machine has less computational power than some other models of computation such as the Turing machine.[2] The computational power distinction means there are computational tasks that a Turing machine can do but a FSM cannot. This is because a FSM's memory is limited by the number of states it has. FSMs are studied in the more general field of automata theory.

## Formalizing FSMs 

Let's consider a simple vending machine where everything costs 20 cents. And let's say that this vending machine only takes nickels and dimes but nothing larger or smaller. Then we can model the state of this vending machine by the amount of money that's been deposited. The start state would be zero cents. And from this date there are two things that can happen. We could put in a nickel, which would make the state five cents or we could put in a dime to take the state to 10 cents. The rest of the transitions are fairly straightforward until we think about what to do if we are in the 15 cent state and someone puts in a dime. We could just count that as 20. But let's say that this machine requires exact change so that a dime would just fall through and come out of the little tray at the bottom of the machine. 

<img src="./outputs/bp7.png" height=600 width=600/>


As you can see, finite state machines are pretty straightforward conceptually. So why talk about them? They have their strengths and weaknesses. Let's start with our strengths. First, finite state machines are very easy to reason about. They are basically self-documenting because they map the logical state of a system directly to the physical state. When a nickel goes into the vending machine the state changes to the state that is five cents bigger than the current one. Next, they are maintainable. If we wanted to tweak this machine so that everything costs a quarter it would be pretty trivial to just add one more state. Which brings us to the weaknesses of the finite state machine. The primary one being that they are easily abused. If they aren't designed well to begin with or if the problem changes you can easily find yourself saying things like, I hadn't considered that. Let's just add another state and this can lead to some sloppy code and sloppy logic. Which in practice means that finite state machines can be very difficult to maintain as the states base gets bigger. 

<img src="./outputs/bp8.png" height=600 width=600/>


<img src="./outputs/bp8.jpg" height=600 width=600/>

If we wanted this machine to also accept pennies (1 cent coin) how many additional states would we have to add?

**We would need one state for each integer between 0 and 25 (which is 26). We already have 6 states. So we would need 20 new states.**

## States For A SelfDrivingCar 

When we're thinking about a simple vending machine it's easy to enumerate the states. Now let's consider the states we may want to model for car driving on a highway. Benny, let's just throw out all the ideas we can come up with and then prune the list afterwards so we can show how we might think about coming up with a finite state machine from scratch. Okay, let's keep it simple first. What happens if we are the only car on the road? I guess we would need a state for normal staying in your lane driving. If we are changing lanes, we want a state to represent that. Or maybe two states since changing lanes to the left is different than to the right. What changes if there's a vehicle in front of us? Well we might want to stay behind it, so we should have a state for that or we might want to pass it. But isn't pass just a lane change to the left and then a lane change to the right? Do we really need in a state for that?  Well, what changes if we had more cars? I could imagine in this situation you might want to slow down so that you could merge into the gap  and then pass the car if it's going slow. We should really have a slow downstate which I guess means we will also need a speed upstate. We should probably add a stop state in case there is some emergency situation. we could do all of that at once by having a keep target speed state. Shouldn't speed just be dictated by the lane you're driving in and the speed limit? What if we just add a prepare lane change state to represent when the ego vehicle is trying to go next to an empty gap in traffic before a lane change? we should probably have a prepare lane change left and to prepare a lane change right, since those really are different maneuvers. we could also turn on the turn signals here. The truth is there isn't a single correct set of states to choose from. On one hand we want to keep our state space as small as possible for maintainability reasons, but on the other hand we want to make sure we have enough logical States to actually represent all the physical states that we care about. As  seen in the discussion we've just had it's not easy to find the perfect set of states for this scenario. 

<img src="./outputs/bp9.png" height=600 width=600/>

## States For A SelfDrivingCar Solution 

In planning this lesson, we knew we wanted to focus on highway driving, but we still debated what states we should use for our finite state machine. We quickly agreed on three, since it seemed to us like these were the bare minimum and that a single lane change wouldn't suffice. And we also agreed to rule out a lot of states because most of these can be thought of as various implementations of the keep lane state. But these last two, prepare lane change left and prepare lane change right, we were unsure about. Eventually, we did decide to use these states for these lessons. So we should probably clarify what we mean by these states. 

Let's start by imagining what vehicle behavior would look like without these two prep states. First, let me explain how this example will work. Up here we have a two-lane highway with traffic driving to the right. This is our self-driving car in blue and other traffic is shown in red. As you watch this animation, you should imagine watching from a helicopter which is flying above the highway at the self-driving car's target speed. These red trails indicate the relative speed of other vehicles. In this situation, the vehicles in lane one are moving faster than we are while the vehicles in lane two are moving slower.
<img src="./outputs/bp10.png" height=600 width=600/>


When we only have three states plus a fourth ready state, the finite state machine looks like this. We'll assume that as we begin watching we are already in the keep lane state. Now before we watch what happens to our vehicle when it uses this finite state machine, let me explain what these states mean. The lane keep state attempts to stay in the current lane by staying near the center line for that lane. So thinking in finite coordinates, we might just say that target d for the vehicle is whatever the d for the lane is. And for the s direction, keep lane state attempts to drive at the vehicle's target speed when that's feasible, but when it's not, it will try to drive at whatever speed is safest for the lane. For lane changes the goal is to move from the initial lane to the target lane. The d behavior is what you might expect to move left or right as appropriate. So the target d is the d for whatever lane is to the left or right of the ego's current lane. For s, the same rules as lane keeping apply. The vehicle will try to drive at the target speed, but if that's not feasible, then it will drive at whatever speed is safe for the initial lane. Now, let's observe the vehicle's behavior as we move forward in time. For the first few timesteps, we see that the ego-vehicle gets closer and closer to vehicle 2.2 while the vehicle in the left lane pas the ego-vehicle. And during this time the vehicle remains in the lane keep state since there isn't really enough gap to make a safe lane change. Note that at this point the ego-vehicle does begin to slow down even though it doesn't change state. That's because the lane keep state contains this behavior. When the target speed is no longer feasible, it will adjust speed to something safe for the lane. At this point, the vehicle has decided that the lane change left is the best move. Now, in the lane change state, you can see the vehicle has moved left in its lane but hasn't adjusted its speed. This is consistent with the rules we defined for lane change behavior. At this point, the vehicle has crossed the dotted line, so its current lane is the left lane. Now it wants to transition back to the lane keep state to stay in this lane, and when it does that, it immediately starts accelerating so it can get to a speed that's safe for this lane. Right now it is moving at the same speed as other traffic in the left lane, which it can continue doing until it passes the vehicles on the right. Even though this worked, there were some problems. First, even when it was clear that there was a gap we could probably move into on the left, we had no way to tell the vehicle to try to get alongside that gap. We had to just wait for the gap to get close to us. Second, this lane change wasn't all that safe. Ideally, we would have been able to get closer to the left lane speed before actually making the lane change. Third, it isn't clear when we would have turned on the turn signal in this scenario. The turn signal is the responsibility of the behavior team and ideally we would like to turn it on a few seconds before we actually start changing lanes. Using this finite state machine, it would have been hard to do that. To address that problem, we can introduce a prepare for lane change state. 

<img src="./outputs/bp11.png" height=600 width=600/>

In this state we do whatever we can to prepare for a lane change left or right, which means in the d directions we still can stay in this current lane but in the s direction we tried to match the position and speed of some gap in one of the adjacent lanes. This is also when we would turn on the appropriate signal. We also changed the finite state machine to look like this. And note that the only way to change lanes is to first prepare for a lane change. Let's see what that would look like. At the first timestep we can decide to prepare for a lane change by tracking this gap over here, which allows us to immediately start slowing down. At this point, the car may begin slowly increasing speed to get closer to the left lane speed, and at this point the vehicle has reached the same speed as the left lane traffic and is in a good position to execute a lane change, which it begins here. From here on, everything looks pretty similar to before. With these added states, we're now able to perform a lane change more safely and efficiently.

## Inputs To Transition Functions

We just saw how the states we choose to use can impact the behavior of the vehicle. But deciding how those states transition and what inputs the transition functions use is crucial to the actual implementation of a finite-state machine. For the example with the vending machine, the only input was the coin. The self-driving car is more complicated. So the question is, what data will we need to pass into our transition function as input. 


<img src="./outputs/bp12.png" height=600 width=600/>

## Behavior Planning Pseudocode

One way to implement a transition function is by generating rough trajectories for each accessible "next state" and then finding the best. To "find the best" we generally use cost functions. We can then figure out how costly each rough trajectory is and then select the state with the lowest cost trajectory.

We'll discuss this in more detail later, but first read carefully through the pseudocode below to get a better sense for how a transition function might work.


***
```
def transition_function(predictions, current_fsm_state, current_pose, cost_functions, weights):
    # only consider states which can be reached from current FSM state.
    possible_successor_states = successor_states(current_fsm_state)

    # keep track of the total cost of each state.
    costs = []
    for state in possible_successor_states:
        # generate a rough idea of what trajectory we would
        # follow IF we chose this state.
        trajectory_for_state = generate_trajectory(state, current_pose, predictions)

        # calculate the "cost" associated with that trajectory.
        cost_for_state = 0
        for i in range(len(cost_functions)) :
            # apply each cost function to the generated trajectory
            cost_function = cost_functions[i]
            cost_for_cost_function = cost_function(trajectory_for_state, predictions)

            # multiply the cost by the associated weight
            weight = weights[i]
            cost_for_state += weight * cost_for_cost_function
         costs.append({'state' : state, 'cost' : cost_for_state})

    # Find the minimum cost state.
    best_next_state = None
    min_cost = 9999999
    for i in range(len(possible_successor_states)):
        state = possible_successor_states[i]
        cost  = costs[i]
        if cost < min_cost:
            min_cost = cost
            best_next_state = state 

    return best_next_state

```
***



Obviously we are glossing over some important details here. Namely: what are these cost functions and how do we create them? We'll talk about that next!

### Create A CostFunction SpeedPenalty 

A key part of getting transitions to happen when we want them to is the design of reasonable cost functions. We want to penalize and reward the right things. We are going to work through an example of one way you might think about designing a cost function. Let's consider how we would design a cost function for vehicle speed. On one hand, we want to get to our destination quickly, but on the other hand, we don't want to break the law. An essential quantity we have to control is the desired velocity of the car. Some velocities are more beneficial, some are even illegal. Let's fill in this graph and try to assign some costs to every velocity. For the sake of simplicity, let's assume that all of the cost functions will have an output between zero and one. We will adjust the importance of each cost function later by adjusting the weights. Let's say the speed limit for the road we are on is marked with **speed_limit** on velocity axis . Well, we know that if we're going well above the speed limit, that should be maximum cost. And maybe we want to set an ideal zero cost speed, **target_speed** that's slightly below the speed limit so that we have some buffer. And then we can think about how much we want to penalize not moving at all. Obviously, not moving is bad, but maybe not as bad as breaking the speed limit, so we would put the cost below 1. To keep it simple, we could just say there is a linear cost between zero and the target speed. And since breaking the law is a binary thing, let's just say any speed greater than or equal the speed limit has maximal cost. And again, we can arbitrarily connect these points with a linear function and the flat maximum cost for anything above the speed limit. Now, in practice, we might actually want to **parametrize some of these quantities so that we could later adjust them until we got the right behavior**. So first, we might define a parameter called **Stop Cost** for the zero-velocity case and a parameter called **buffer velocity** which would probably be a few miles per hour. Then, our overall cost function has three domains. If we're going less than the target speed, the cost function would look like as in left of image. If we are above the speed limit, the cost is just one. And if we are between, the cost would look like as in bottom right of image. 


<img src="./outputs/bp13.png" height=600 width=600 />

## Example Cost Function - Lane Change Penalty

<img src="./outputs/bp14.png" height=600 width=600 />


In the image above, the blue self driving car (bottom left) is trying to get to the goal (gold star). It's currently in the correct lane but the green car is going very slowly, so it considers whether it should perform a lane change (LC) or just keep lane (KL). These options are shown as lighter blue vehicles with a dashed outline.

If we want to design a cost function that deals with lane choice, it will be helpful to establish what the relevant variables are. In this case, we can define:

$\Delta s = s_G - s $ how much distance the vehicle will have before it has to get into the goal lane.

$\Delta d = d_G - d_{LC/KL} $ the lateral distance between the goal lane and the options being considered. In this case $\Delta d_{KL} = d_G - d_{KL} $ would be zero and $\Delta d_{LC} = d_G - d_{LC} $ would not.

Before we define an actual cost function, let's think of some of the properties we want it to have...

First, thinking only about delta d: Would we prefer the absolute value of $\Delta d $ to be big or small?
    * Small

Now let's think about how s factors into our considerations of lane cost. Should costs associated with lane change be more important when we are far from the goal (in s coordinate) or close to the goal?
    * Close

So we want a cost function that penalizes large $|\Delta d| $ and we want that penalty to be bigger when $\Delta s $ is small.

Furthermore, we want to make sure that the maximum cost of this cost function never exceeds one and that the minimum never goes below zero.

Which of the following proposals meets these criteria?

Option 1: $\text{cost} = |\Delta d| + \frac{1}{\Delta s} $
 

Option 2: $\text{cost} = \frac{|\Delta d|}{\Delta s} $


Option 3: $\text{cost} = 1 - e^{- \frac{|\Delta d|}{\Delta s}} $

    * Option 3
    
In this example, we found that the ratio $\LARGE \frac{|\Delta d|}{\Delta s} $  was important. If we call that ratio $\text{x} $ we can then use that ratio in any function with bounded range. These functions tend to be useful when designing cost functions. These types of functions are called **Sigmoid Functions**. You can learn more in the Wikipedia article if you're interested.

## Sigmoid

[Wikipedia](https://en.wikipedia.org/wiki/Sigmoid_function)

A sigmoid function is a mathematical function having a characteristic "S"-shaped curve or sigmoid curve. Often, sigmoid function refers to the special case of the logistic function shown in the first figure and defined by the formula

${\displaystyle S(x)={\frac {1}{1+e^{-x}}}={\frac {e^{x}}{e^{x}+1}}.} $

<img src="./outputs/sigmoid1.png" height=300  width=300 /> 
**[The logistic curve](https://en.wikipedia.org/wiki/Logistic_curve)**

<img src="./outputs/sigmoid2.png" height=300  width=300 />
**[Plot of the error function](https://en.wikipedia.org/wiki/Error_function)**

Other examples of similar shapes include the Gompertz curve (used in modeling systems that saturate at large values of x) and the ogee curve (used in the spillway of some dams). Sigmoid functions have domain of all real numbers, with return value monotonically increasing most often from 0 to 1 or alternatively from −1 to 1, depending on convention.

A wide variety of sigmoid functions have been used as the activation function of artificial neurons, including the logistic and hyperbolic tangent functions. Sigmoid curves are also common in statistics as cumulative distribution functions (which go from 0 to 1), such as the integrals of the logistic distribution, the normal distribution, and Student's t probability density functions.


A sigmoid function is a **bounded, differentiable, real function** that is defined for all real input values and has a **non-negative derivative** at each point.

In general, a sigmoid function is real-valued, monotonic, and differentiable having a non-negative first derivative which is bell shaped. A sigmoid function is constrained by a pair of horizontal asymptotes as ${\displaystyle x\rightarrow \pm \infty } $

#### Examples of Sigmoid functions

* Logistic function
${\displaystyle f(x)={\frac {1}{1+e^{-x}}}}$

<img src="./outputs/sigmoid.png" height=600  width=600 /> 

* hyperbolic tangent (shifted and scaled version of Logistic, above)
$ {\displaystyle f(x)=\tanh x={\frac {e^{x}-e^{-x}}{e^{x}+e^{-x}}}}$ 

* arctangent function
$ {\displaystyle f(x)=\arctan x} $

* Gudermannian function
$ {\displaystyle f(x)=\operatorname {gd} (x)=\int _{0}^{x}{\frac {1}{\cosh t}}\,dt} $

* Error function
$ {\displaystyle f(x)=\operatorname {erf} (x)={\frac {2}{\sqrt {\pi }}}\int _{0}^{x}e^{-t^{2}}\,dt} $

* Generalised logistic function
$ {\displaystyle f(x)=(1+e^{-x})^{-\alpha },\quad \alpha >0} $

* Smoothstep function
$ {\displaystyle f(x)={\begin{cases}\left(\int _{0}^{1}{\big (}1-u^{2}{\big )}^{N}\ du\right)^{-1}\int _{0}^{x}{\big (}1-u^{2}{\big )}^{N}\ du\quad &|x|\leq 1\\\operatorname {sgn} (x)&|x|\geq 1\\\end{cases}}\,\quad N\geq 1} $

* Specific algebraic functions
$ {\displaystyle f(x)={\frac {x}{\sqrt {1+x^{2}}}}}$

<img src="./outputs/sigmoid3.png" height=600  width=600 /> 


The integral of any continuous, non-negative, "bump-shaped" function will be sigmoidal, thus the cumulative distribution functions for many common probability distributions are sigmoidal. One such example is the error function, which is related to the cumulative distribution function (CDF) of a normal distribution.

### Quizz: Implement a Cost Function in C++

In the previous quizzes, you designed a cost function to choose a lane when trying to reach a goal in highway driving:

$\text{cost} = 1 - e^{- \frac{|\Delta d|}{\Delta s}}$

Here, $\Delta d$ was the lateral distance between the goal lane and the final chosen lane, and $\Delta s$ was the longitudinal distance from the vehicle to the goal.

In this quiz, implement the cost function in C++, but with one important change. The finite state machine we use for vehicle behavior also includes states for planning a lane change right or left (PLCR or PLCL), and the cost function should incorporate this information. The following four are inputs to the function:

1. Intended lane: the intended lane for the given behavior. For PLCR, PLCL, LCR, and LCL, this would be the one lane over from the current lane.

2. Final lane: the immediate resulting lane of the given behavior. For LCR and LCL, this would be one lane over.

3. The $\Delta s$ distance to the goal.

4. The goal lane.

Objective  in the implementation will be to modify $|\Delta d|$ in the equation above so that it satisifes:

* $|\Delta d|$ is smaller as both intended lane and final lane are closer to the goal lane.

* The cost function provides different costs for each possible behavior: KL, PLCR/PLCL, LCR/LCL.

* The values produced by the cost function are in the range 0 to 1.

```
#include <functional>
#include <iostream>
#include "cost.h"
#include "cmath"


using namespace std;

float goal_distance_cost(int goal_lane, int intended_lane, int final_lane, float distance_to_goal) {
    /*
    The cost increases with both the distance of intended lane from the goal
    and the distance of the final lane from the goal. The cost of being out of the 
    goal lane also becomes larger as vehicle approaches the goal.
    */
    
    //Replace cost = 0 with an appropriate cost function.
    float del_s=distance_to_goal;
    //float del_d=(goal_lane-final_lane)+(goal_lane-intended_lane);
    float del_d=2.0*goal_lane - intended_lane - final_lane;
    float cost = 1-exp(-abs(del_d)/del_s);
    
    
    return cost;
}

```

### Quiz: Implement a Second Cost Function in C++

In most situations, a single cost function will not be sufficient to produce complex vehicle behavior. In this quiz, implement one more cost function in C++.The goal with this quiz is to create a cost function that would make the vehicle drive in the fastest possible lane, given several behavior options. We will provide the following four inputs to the function:

1. Target speed: Currently set as 10 (unitless), the speed at which you would like the vehicle to travel.
2. Intended lane: the intended lane for the given behavior. For PLCR, PLCL, LCR, and LCL, this would be the one lane over from the current lane.
3. Final lane: the immediate resulting lane of the given behavior. For LCR and LCL, this would be one lane over.
4. A vector of lane speeds, based on traffic in that lane: {6, 7, 8, 9}.

The task in the implementation will be to create a cost function that satisifes:

1. The cost decreases as both intended lane and final lane are higher speed lanes.
2. The cost function provides different costs for each possible behavior: KL, PLCR/PLCL, LCR/LCL.
3. The values produced by the cost function are in the range 0 to 1.


```
#include <functional>
#include <iostream>
#include "cost.h"
#include "cmath"


using namespace std;

float inefficiency_cost(int target_speed, int intended_lane, int final_lane, vector<int> lane_speeds) {
    /*
    Cost becomes higher for trajectories with intended lane and final lane that have traffic slower than target_speed.
    */

    float intended_lane_speed=target_speed-lane_speeds[intended_lane];
    float final_lane_speed=target_speed-lane_speeds[final_lane];
    float cost=(intended_lane_speed+final_lane_speed)/target_speed;
    
    return cost;
}

```

## CostFunction Design and  WeightTweaking 

Designing cost functions is difficult and getting them all to cooperate to produce reasonable vehicle behavior is hard. Some of the difficulties associated with cost functions design include solving new problems without unsolving old ones. When you're working on a self-driving car, you may find that the vehicle is behaving reasonably well except for some particular situations. Maybe it's not being aggressive enough about making left turns at traffic lights. So, in an effort to solve this problem, you either add new cost functions, tweak existing ones, or modify the weights. But every time you do, there's a chance that you will introduce some breaking change into something that already works. In practice, we solve this through regression testing, where we define some set of situations, each of which has an expected behavior. Then, whenever we make a change, we simulate the vehicle in all of our test cases and make sure that it still behaves as expected. We won't say more about testing here, but it is an important part of developing software in a safety-critical application. The next difficulty is balancing costs of drastically different magnitudes. Because, yes, we want to get to our destination efficiently, but if we are in a situation where safety is an issue, we want to solve that problem and not think about efficiency at all. One way to do that is to have weights which reflect the type of problem the cost function addresses. So we want to most heavily penalize any behavior which simply isn't possible due to physics, then we want to think about safety, legality, comfort. And only once those are satisfied do we want to think about efficiency. But we also may want to adjust the relative importance of these weights depending on situation. If a light turns red, for example, legality becomes a much more relevant concern than when we engage in normal highway driving. And this leads us to our last difficulty, reasoning about individual cost functions. Ideally, each cost function will serve a very specific responsibility, which is something we didn't do in our earlier example of a speed cost function. We were trying to balance our desire to drive quickly, which has to do with efficiency, with our desire to not exceed the speed limit, which is legality. In practice, we might want to define several cost functions associated with vehicle speed. In which case we might have a binary cost function which just checks to see if we are breaking the speed limit and the continuous cost function which pulls us towards our target speed. By assigning each cost function to a very specific role, like safety versus legality versus efficiency, we can then standardize the output of all cost functions to be between -1 and 1. Additionally, it's helpful to parametrize whenever possible. This allows us to use some parameter optimization technique like gradient descent along with our set of regression tests to programmatically tweak our cost functions. Finally, thinking in terms of vehicle state is helpful. The things we can indirectly control about our vehicle are its position, velocity, and acceleration. It can be helpful to keep these in mind when coming up with cost functions. 


<img src="./outputs/bp15.png" height=600 width=600 />




Let's walk through an example. Say we want to think about the following classes of cost functions. And to make it easier, to keep everything straight, let's think in terms of position, velocity, and acceleration. The binary "Are we exceeding the speed?" cost function would go legality, then the cost function that wanted to keep us close to the speed limit would go efficiency. And now, instead of being that weird discontinuous graph we made earlier, this could just be some parabola like this. And even though the cost of this is low even for speeds that exceed the speed limit, that's OK because we have our binary cost function which would prevent that behavior. 
<img src="./outputs/bp16.png" height=600 width=600 />



Continuing to think about speed, we also might want to try to drive at a speed that's close to the average speed of traffic, for safety reasons, even if that speed is above or below the speed limit. And that would go in safety.

In the position column, we'd have an obvious feasibility concern which is collision avoidance. We can't drive somewhere if there is already a car there. this goes in feasibility section

Then, for safety reasons, we would want the buffer distance (safety), which tries to keep us far from other vehicles, and the cost function which checks to make sure we are driving on the road(legality) near the center of our lane (comfort) and in a lane that's close to our desired goal lane (efficiency). For acceleration, we'd first want to make sure we only consider behaviors that the car can execute(feasibility), and then we'd want to avoid having any rapid changes in acceleration because those are perceived as uncomfortable(comfort). This is also known as jerk. 
<img src="./outputs/bp17.png" height=600 width=600 />

Consider a merge onto a highway, for example. This is a potentially dangerous situation where we really want to get up to traffic speed as quickly as possible. So this cost function may become more relevant than it normally is. But we also want to make sure that we yield if there isn't really a gap, so we want to make sure avoid collision cost function and buffer distance one are weighted sufficiently high. 
<img src="./outputs/bp18.png" height=600 width=600 />


And we can compare these merge priorities to a different situation. For example, a car approaching a green light that suddenly turns yellow. In this situation, we'd probably want to boost the weights associated with legality and would probably need to add  a whole new cost function for **obeying traffic rules**. Now that doesn't fall neatly into the position,  velocity, or acceleration classes,  so We just put it out of table . 


<img src="./outputs/bp19.png" height=600 width=600 />



If this is all starting to feel like it's getting pretty complex, well, you're right. It's pretty hard to avoid this exploding complexity when using finite state machines. Partially that's because of the finite state machine itself, but we're also trying to solve a very hard problem and some complexity is unavoidable no matter what solution approach you take.

## Cost Function Matching
Consider the following cost functions. What purpose does each serve?

Cost Function 1: $\begin{cases} 1 & \ddot{s} \geq a\_{\text{max}} \\ 0 & \ddot{s} < a\_{\text{max}} \end{cases} $

Cost Function 2: $\begin{cases} 1 & d \geq d\_{\text{max}} \\ 1 & d \leq d\_{\text{min}} \\ 0 & d\_{\text{min}} < d < d\_{\text{max}} \end{cases} $ 

Cost Function 3: $\begin{cases} 1 & \dot{s} \geq v\_{\text{speed limit}} \\ 0 & \dot{s} < v\_{\text{speed limit}} \end{cases} $

Cost Function 4: $\frac{1}{1 + e^{-(d - d\_{\text{lane center}})^2}} $

Cost Function 5: $(\text{lane number} - \text{target lane number}) ^ 2 $ 

<img src="./outputs/bp20.png" height=600 width=600/>

## Scheduling ComputeTime 

In the very beginning of the lesson you have already seen this diagram. 

<img src="./outputs/bp21.png" height=500 width=600 />

By now you might be able to guess why the Behavior Module updates on a lower frequency than for example the Trajectory Module. This is due to the fact that the high level decisions made in the behavior module spend a longer time horizon and just don't change as frequently. But the Trajectory Module does still count on our decisions and it's important that the over all system architecture doesn't allow for a comparatively slow module like the behavior planner to clock up the proper functioning of the other faster components. Let's take a second to talk about what's known as a **scheduling problem** and how it can be handled in the self-driving car. This diagram shows what happens during two processing cycles of the Behavior Module.
<img src="./outputs/bp22.png" height=600 width=600 />

As you can see, the Prediction Module updates with a higher frequency than Behavior. Trajectory is even higher. And so on. But focus your attention on what happens after behavior has completed its first cycle. To begin its second cycle, the behavior module needs data from prediction and localization. 
<img src="./outputs/bp23.png" height=600 width=600 />


For localization, it's easy in theory since at this instant it will have some fresh data and behavior could just use that. But what about for prediction. It's actually right in the middle of an update cycle at this instant. Should behavior just wait until prediction is done? No. If we start waiting then we block up the pipeline for downstream components. The answer is to use data from here and accept that it's a little stale. 

<img src="./outputs/bp24.png" height=600 width=600 />

When you implement your half planner for the final project, we will provide you with the code that handles all of this. But it's worth mentioning that this is how it's done. 

## End of Lesson Quiz

## Implement a Behavior Planner

In this exercise we will implement a behavior planner and cost functions for highway driving. The planner will use prediction data to set the state of the ego vehicle to one of 5 values and generate a corresponding vehicle trajectory:

* <span style="color:red;">"KL"</span> - Keep Lane

* <span style="color:red;">"LCL" / "LCR"</span>- Lane Change Left / Lane Change Right
* <span style="color:red;">"PLCL" / "PLCR"</span> - Prepare Lane Change Left / Prepare Lane Change Right

The objective of the quiz is to navigate through traffic to the goal in as little time as possible. Note that the goal lane and s value, as well as the traffic speeds for each lane, are set in <span style="color:red;">main.cpp</span> below. Since the goal is in the slowest lane, in order to get the lowest time, you'll want to choose cost functions and weights to drive in faster lanes when appropriate. We've provided two suggested cost functions in <span style="color:red;">cost.cpp</span>.

### Instructions

* **Implement the <span style="color:red;">choose_next_state</span> method in the <span style="color:red;">vehicle.cpp</span> class**. We will use the Behavior Planning Pseudocode concept as a guideline for the implementation. In this quiz, there are a couple of small differences from that pseudocode: we'll be returning a best trajectory corresponding to the best state instead of the state itself. Additionally, the function inputs will be slightly different in this quiz than in the classroom concept. For this part of the quiz, several useful functions are provided:
    1. <span style="color:red;">successor_states()</span> - Uses the current state to return a vector of possible successor states for the finite state machine.
    2. <span style="color:red;">generate_trajectory(string state, map<int, vector<Vehicle>> predictions)</span> - Returns a vector of Vehicle objects representing a vehicle trajectory, given a state and predictions. Note that trajectory vectors might have size 0 if no possible trajectory exists for the state.
    3. <span style="color:red;">calculate_cost(Vehicle vehicle, map<int, vector<Vehicle>> predictions, vector<Vehicle> trajectory)</span> - Included from cost.cpp, computes the cost for a trajectory.
    
* **Choose appropriate weights for the cost functions in <span style="color:red;">cost.cpp</span> to induce the desired vehicle behavior**. Two suggested cost functions have been implemented based on previous quizzes, but additional experiment with new cost functions can be tried. The <span style="color:red;">get_helper_data(const Vehicle & vehicle, const vector<Vehicle> & trajectory, const map<int, vector<Vehicle>> & predictions)</span> function in <span style="color:red;">cost.cpp</span> provides some preprocessing of vehicle data that may be useful in the cost functions. See if we can get the vehicle to move into a fast lane for a portion of the track and then move back to reach the goal!

* Hit Test Run and see how the car does! How fast can you get to the goal?

## Extra Practice
Provided in one of the links below is a zip file <span style="color:red;"> python\_3\_practice </span> [link](./python-3-practice.zip), which is the same problem written in Python 3 - you can optionally use this file for additional coding practice. In the <span style="color:red;">python\_3_solution </span> [link](./python-3-solution.zip), the solution is provided as well. If you get stuck on the quiz see if you can convert the python solution to C++ and pass the classroom quiz with it. You can run the python quiz with <span style="color:red;">python simulate\_behavior.py </span>.

## Solution

Solution to the quiz is in directory [src](./src)

To run the code, do following steps:
1. mkdir build
2. cd build
3. cmake ..
4. make
5. ./BehaviorPlanning


### Notes:
* goal_distance_cost function has a expo term multiplied by 2
$\text{cost} = 1 - 2*e^{\frac{(-(|(2.0*vehicle.goal_lane - data["intended\_lane"] - data["final\_lane"]|}{distance}} $;
So, cost will be between +1 and -1

* weight goal_dist vs efficiency is 10:1 works for given testcase, vehicles gets to goal of 320 in 32 seconds

* when speed limit is increased, the time taken to goal is increased to 46 units!! vehicles moves to goal lane (slowest lane) quite early and stays there, never transitions to fast lane.

## Conclusion

In this lesson we learned about FSM which are a solution to behavior planning problem.FSM does well when the state space is small such as highway driving.For more complex scenario like urban driving other (??) approaches might be more suitable.